In [49]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from controller.ppo_vanilla import Agent
from vision.vAE import VAE
from data.car_racing import CarRacingWrapper
from memory import Memory
import numpy as np
import tensorflow as tf

### Understanding **tf.element_spec**

In [2]:
dummy = tf.data.Dataset.from_tensor_slices(
    (np.zeros(999*35).reshape(999,35).astype(np.float32), np.zeros(999*32).reshape(999,32).astype(np.float32))
)
dummy.element_spec

(TensorSpec(shape=(35,), dtype=tf.float32, name=None),
 TensorSpec(shape=(32,), dtype=tf.float32, name=None))

In [3]:
t_spec = dummy.element_spec

In [4]:
ds = tf.data.experimental.load(
    os.getcwd() + '/saved_data/z_vae160',
    compression='GZIP',
    element_spec=(tf.TensorSpec(shape=(999, 35), dtype=tf.float32), tf.TensorSpec(shape=(999, 32), dtype=tf.float32))
)
ds

<_LoadDataset shapes: ((999, 35), (999, 32)), types: (tf.float32, tf.float32)>

In [5]:
ds = ds.shuffle(1).batch(4).prefetch(tf.data.experimental.AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 999, 35), (None, 999, 32)), types: (tf.float32, tf.float32)>

In [8]:
len(ds)

63

In [7]:
ds = ds.map(lambda x, y: (tf.cast(x, tf.float64), tf.cast(y, tf.float64)))

In [10]:
import time

def train(model, data, epochs):
    epoch_losses = []
    for e in range(epochs):
        t = time.time()
        batch_losses = []
        data = data.shuffle(1)
        
        for (x, y) in data.take(16):
            state = model.get_zero_hidden_state(x)
            batch_losses.append(model.train_op(x, y, state))
            
        epoch_losses.append(np.mean(batch_losses))
        print(f"Epoch: {e} done - Loss: {epoch_losses[-1]} - Time: {time.time()-t}")

    model.save_weights(os.getcwd() + '/160model_v2')
        
    return batch_losses

In [11]:
mem = Memory()
mem.load_weights(os.getcwd() + '/160model_v2')
loss = []

In [ ]:
loss.append(train(mem, ds, 16))
print(loss[-1])

## Visualizing performance

In [17]:
import matplotlib.pyplot as plt

In [35]:
vision = VAE(results_dir=os.getcwd()[:-6] + 'vision', load_model=True)

loading model from /Users/floyd/Documents/Studium/Coxi6/DRL/World-Model-CarRacing-with-PPO/world_model/vision/models


In [36]:
def dream(memory, n=2):
    fig, ax = plt.subplots(ncols=2, nrows=n)

    for i, (x, y) in enumerate(ds.take(n)):
        
        x = x.numpy()
        x = tf.cast(x[0,0,:], tf.float64)
        
        state = memory.get_zero_hidden_state(x)
        mix_params, _, _ = memory(x, state, temperature=1.0)
        pred = memory.get_y_pred(mix_params)
        
        ax[i, 0].imshow(tf.squeeze(vision.decoder(pred[0,0])))
        ax[i, 0].set_title("Dream")
        ax[i, 0].axis("off")
        
        ax[i, 1].imshow(tf.squeeze(vision.decoder(y.numpy()[0,0].reshape(1,32))))
        ax[i, 1].set_title("Truth")
        ax[i, 1].axis("off")

    plt.show()

In [ ]:
dream(mem, n=2)